Imports

In [ ]:
import pandas as pd
import numpy as np
import math
import random as rd
import matplotlib.pyplot as plt
from datetime import datetime
import calendar
import locale

locale.setlocale(locale.LC_TIME, "en_US.utf8") # English US

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Import the dependent data in from\
`Fitness & Rec Survey 2024!!! (Responses) - Form Responses.csv`



In [ ]:
# Data paths
d_path = 'drive/Shareddrives/Merrimack Gym Congestion Analysis/Fitness & Rec Survey 2024!!! (Responses) - Form Responses 1 (2).csv'
i_fall_path = 'drive/Shareddrives/Merrimack Gym Congestion Analysis/Swipe 8.28-12.8.23.xlsx'
i_spring_path = 'drive/Shareddrives/Merrimack Gym Congestion Analysis/Swipe 1.17-4.16.24.xlsx'

# Read in the data
d_df = pd.read_csv(d_path)
i_fall_df = pd.read_excel(i_fall_path)
i_spring_df = pd.read_excel(i_spring_path)

d_df
i_fall_df
i_spring_df


In [ ]:
# @title Affiliation

from matplotlib import pyplot as plt
import seaborn as sns
i_spring_df.groupby('Affiliation').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

**Data Cleaning** \

Two stages


*   Remove student identifiers, Name & Email Addresses, from All three datasets; to anonymize the data
*   Drop rows in dependent data where gener != Male or Female
*   Drop rows in independent data where gener != Male or Female
*   Drop rows in independent data for breaks where Date is:
    *  Fall
      *   09/04/2023
      *   10/09/2023
      *   10/10/2023
      *   11/10/2023
      *   11/22/2023
      *   11/23/2023
      *   11/24/2023      
      *   11/25/2023
      *   11/26/2023
    *  Spring
      *   02/19/2024
      *   03/04/2024
      *   03/05/2024
      *   03/06/2024
      *   03/07/2024
      *   03/08/2024
      *   03/09/2024
      *   03/28/2024
      *   03/29/2024
      *   03/30/2024
      *   03/31/2024
      *   04/01/2024

      









In [ ]:
# Drop rows in dependent data where gener != Male or Female
valid_gender = ['Male', 'Female']
d_df = d_df[d_df['Gender'].isin(valid_gender) == True]

# Drop rows in independent data where gener != Male or Female
i_fall_df = i_fall_df[i_fall_df['Gender'].isin(valid_gender) == True]
i_spring_df = i_spring_df[i_spring_df['Gender'].isin(valid_gender) == True]

# Drop rows in independent data for breaks where Date is: Fall
invalid_fall_dates = ['09/04/2023', '10/09/2023', '10/10/2023', '11/10/2023', '11/22/2023', '11/23/2023', '11/24/2023', '11/25/2023', '11/26/2023']
i_fall_df = i_fall_df[i_fall_df['Date'].str.contains('|'.join(invalid_fall_dates)) != True]

# Drop rows in independent data for breaks where Date is: Spring
invalid_spring_dates = ['02/19/2024', '03/04/2024', '03/05/2024', '03/06/2024', '03/07/2024', '03/08/2024', '03/09/2024', '03/28/2024', '03/29/2024', '03/30/2024', '03/31/2024', '04/01/2024']
i_spring_df = i_spring_df[i_spring_df['Date'].str.contains('|'.join(invalid_spring_dates)) != True]

# Drop the "Name" column in independent data.
i_fall_df = i_fall_df.drop(['Name'], axis=1, errors='ignore')
i_spring_df = i_spring_df.drop(['Name'], axis=1, errors='ignore')

# Drop the "Email Address" column in dependent data.
d_df = d_df.drop('Email Address', axis=1, errors='ignore')

d_df

**Merging**

Merge the i_fall and i_spring dataframes together. \
This will be used to predict total year congestion.

In [ ]:
# Merge the i_fall_df && i_spring_df dataframes
i_data = pd.concat([i_fall_df, i_spring_df])
i_data

***Graph*** \
**Average Male vs Female Weekly Swipe Frequency**  
*Independent Data Only*  

Graph depicting dependent data in Fall and Spring where Male and Female's are compared based on the day they swipe in.

Graphs needed: Ratio of Male's to Female's based on independent swipe data.




In [ ]:
df = i_data
season = "Data: 2023-2024"

Male = [[], [], [], [], [], [], []]
Female = [[], [], [], [], [], [], []]

# Dictionaries to hold daily swipes, will compute the average over a day afterwads.
Male_D = {}
Female_D = {}

def getDate(key):
  # https://www.geeksforgeeks.org/python-program-to-find-day-of-the-week-for-a-given-date/
  # Convert the input date string to a datetime object
  #   given_date = datetime.datetime.strptime(key, '%m/%d/%Y %H:%M%p')
  given_date = datetime.strptime(key, '%m/%d/%Y') # Only looking at DD/MM/YYYY
  # Use isoweekday() to get the weekday (Monday is 1 and Sunday is 7)
  return given_date.weekday()  # Convert (Monday is 1 and Sunday is 7)

# Adds to the correct gender array based on day of week
def func(d):
    gender = d['Gender']
    date = d['Date'].split(" ")[0] # Get only the DD/MM/YYYY portion of the date.
    if gender == 'Male':
        if date in Male_D:
          Male_D[date] = Male_D[date] + 1
        else:
          Male_D[date] = 1

    elif gender == 'Female':
        if date in Female_D:
          Female_D[date] = Female_D[date] + 1
        else:
          Female_D[date] = 1

# Populate Male_D and Female_D
df.apply(func, axis=1)

# Add all of the days from each dictionary to Male and Female based on their respective day of the week
for key, val in Male_D.items():
  date = getDate(key)
  Male[date].append(val)

for key, val in Female_D.items():
  date = getDate(key)
  Female[date].append(val)

# Flatten and Average Male and Female
Male_Flat = [sum(x)/len(x) for x in Male]
Female_Flat = [sum(x)/len(x) for x in Female]

# Create the graph's and plot
plotdata = pd.DataFrame({ "Male": Male_Flat, "Female": Female_Flat }, index=["Monday", "Tuseday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"])
plotdata.plot(kind="bar",figsize=(15, 8))
plt.title(f"{season} Average Male vs Female Weekly Swipe Frequency")

plt.xlabel("Day of the week")

plt.ylabel("# of Swipes")

plt.show()


***Graph*** \
**Male vs Female Average Daily Swipe Time**  
*Independent Data Only*  

Graph depicting dependent data in Fall and Spring where Male and Female's are compared based on the day they swipe in.

Graphs needed: Ratio of Male's to Female's based on independent swipe data.




In [ ]:
df = i_data
season = "Data: 2023-2024"

        # List of 7 dictionaries
        # Each dictionary represents one day of the week 0 (Monday) - 6 (Sunday)
        #   Inside each dictionary contains a key of hours to # of swipes
Male = [{}, {}, {}, {}, {}, {}, {}]
Female = [{}, {}, {}, {}, {}, {}, {}]

Day_Counts = [[], [], [], [], [], [], []] # Represents unique days Each dictionary represents one day of the week 0 (Sun) - 6 (Sat)
                                          # Used later to compute the average swipes on a given day
def getDayOfWeek(key):
  # Convert the input date string to a datetime object, and set the AM/PM to capital for the datetime parser.
  key.replace("am", "AM")
  key.replace("pm", "PM")
  given_date = datetime.strptime(key, '%m/%d/%Y %I:%M%p')
  return given_date.weekday()  # Convert Sunday from 6 to 0

def getHourOfDay(key):
  # Convert the input date string to a datetime object, and set the AM/PM to capital for the datetime parser.
  key.replace("am", "AM")
  key.replace("pm", "PM")
  given_date = datetime.strptime(key, '%m/%d/%Y %I:%M%p')
  return given_date.strftime("%I%p")  # Convert Sunday from 6 to 0

def getDate(key):
  # Convert the input date string to a datetime object, and set the AM/PM to capital for the datetime parser.
  key.replace("am", "AM")
  key.replace("pm", "PM")
  given_date = datetime.strptime(key, '%m/%d/%Y %I:%M%p')
  return given_date.strftime("%m/%d/%Y")  # Convert to date

# Adds to the correct gender array based on day of week
def func(d):
    gender = d['Gender']
    date = d['Date'] # Get only the DD/MM/YYYY HH portion of the date.

    day_of_week = getDayOfWeek(date) # Gets the day of week (0 (Sun) - 6 (Sat))
    hour = getHourOfDay(date) # Gets the hour of the day from date
    date = getDate(date)

    # Add the date to Day_Counts if it does not contain currentely
    if date not in Day_Counts[day_of_week]:
      Day_Counts[day_of_week].append(date)

    if gender == 'Male':
        Male_D = Male[day_of_week]
        if hour in Male_D:
          Male_D[hour] = Male_D[hour] + 1
        else:
          Male_D[hour] = 1

    elif gender == 'Female':
        Female_D = Female[day_of_week]
        if hour in Female_D:
          Female_D[hour] = Female_D[hour] + 1
        else:
          Female_D[hour] = 1

# Populate Male_D and Female_D
df.apply(func, axis=1)

# Compute the averages of Male_D and Female_D for each day of the week
for i in range(len(Male)):
  num_of_days = len(Day_Counts[i])
  for key, value in Male[i].items():
    Male[i][key] = value / num_of_days

for i in range(len(Female)):
  num_of_days = len(Day_Counts[i])
  for key, value in Female[i].items():
    Female[i][key] = value / num_of_days


plt.tight_layout() # Assign a tigh layout for the plots
# Plot the averages constructed
fig, (ax1, ax2) = plt.subplots(2, figsize=(15, 8))
fig.suptitle(f"{season} Male vs Female Average Daily Swipe Time")
fig.tight_layout()

ax1.set_title("Male")
ax2.set_title("Female")

#ax1.plot(kind="line",figsize=(15, 8))
#ax2.plot(kind="line",figsize=(15, 8))

#ax1.plot(np.array([1, 2, 3, 4, 5]))

dates = ['06AM', '07AM', '08AM', '09AM', '10AM', '11AM', '12PM', '01PM', '02PM', '03PM', '04PM', '05PM', '06PM', '07PM', '08PM', '09PM', '10PM']
weekdays = ["Monday", "Tuseday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

# Append all Weekday Male Averages to the graph
for day in range(len(Male)):
  # Add all of the dates to the image
  swipe_count = [Male[day][x] if x in Male[day] else 0.0 for x in dates]
  ax1.plot(dates, swipe_count, '-', label=weekdays[day]) # this will show date at the x-axis

# Append all Weekday Male Averages to the graph
for day in range(len(Female)):
  # Add all of the dates to the image
  swipe_count = [Female[day][x] if x in Female[day] else 0.0 for x in dates]
  ax2.plot(dates, swipe_count, '-', label=weekdays[day]) # this will show date at the x-axis

# Legend
ax1.legend()
ax2.legend()

fig.show()

Male_Swipe = Male
Female_Swipe = Female


***Graph*** \
**Male vs Female Gym Zone Usage**  
*Dependent Data Only*  

Goal of this graph is to create a bar graph chart to list out each of the seven zones in the gym. \
Cardio (front room),	Cables Area (middle room),	Weight Room (back room), Studio 1 (front), Spin Studio,	Studio 3 (located in Weight Room),	Indoor Track

And determine if Gender effects the congestion of these areas. \
Based on the survey data.





In [ ]:
df = d_df

# Represents our estimated distribution of how utilized a zone is when determining priority
# This can be changed
weights = [0.5, 0.30, 0.15, 0.05, 0.0, 0.0, 0.0]

# Seven zones of the gym
# Cardio (front room),	Cables Area (middle room),	Weight Room (back room), Studio 1 (front), Spin Studio,	Studio 3 (located in Weight Room),	Indoor Track
zone_map = ['Cardio (front room)',	'Cables Area (middle room)',	'Weight Room (back room)', 'Studio 1 (front)', 'Spin Studio',	'Studio 3 (located in Weight Room)',	'Indoor Track']
Male = [0, 0, 0, 0, 0, 0, 0]
Female = [0, 0, 0, 0, 0, 0, 0]

# Function to determine the top choices from a survey result
def func(d):
  order = [d['When I use the MC Fitness Center I spend most of my time: [1st choice]'],
           d['When I use the MC Fitness Center I spend most of my time: [2nd choice]'],
           d['When I use the MC Fitness Center I spend most of my time: [3rd choice]'],
           d['When I use the MC Fitness Center I spend most of my time: [4th choice]'],
           d['When I use the MC Fitness Center I spend most of my time: [5th choice]'],
           d['When I use the MC Fitness Center I spend most of my time: [6th choice]'],
           d['When I use the MC Fitness Center I spend most of my time: [7th choice]']
           ]

  # Map the indexes of the top priorities for a user
  # TODO, Handle NAN
  map = []
  for i in range(len(order)):
    if not (type(order[i]) == float and math.isnan(order[i])):
      map.append(zone_map.index(order[i]))

  #map = [zone_map.index(first), zone_map.index(second), zone_map.index(third), zone_map.index(fourth), zone_map.index(fifth), zone_map.index(sixth), zone_map.index(seventh)]

  # If male add the male weights
  if d['Gender'] == 'Male':
    for i in range(len(map)):
      Male[map[i]] = Male[map[i]] + (weights[i])

  # Othwise, add the female weigths
  elif d['Gender'] == 'Female':
    for i in range(len(map)):
      Female[map[i]] = Female[map[i]] + (weights[i])

# Populate Male_D and Female_D
df.apply(func, axis=1)

# Create the figure
fig, (ax1) = plt.subplots(figsize=(15, 8))
fig.suptitle('Survey: Male vs Female Gym Zone Usage')
fig.tight_layout()

# Create the graph's and plot
plotdata = pd.DataFrame({ "Male": Male, "Female": Female }, index=zone_map)
plotdata.plot(kind='bar', ax=ax1)

#ax1.plot(kind="bar",figsize=(15, 8))
#plt.title(f"{season} Average Male vs Female Weekly Swipe Frequency")

fig.show()

male_congestion_data = Male
female_congestion_data = Female

***Graph*** \
**Students Who Believe Gym is Congested**  
*Dependent Data Only*  

Goal of this graph is to display from the survey how many students believe the gym is congested.





In [ ]:
df = d_df

labels = ["no, it is always occupied and I have to wait 10 minutes or longer", "sometimes, I have to wait several minutes", "yes, I can use it and don't have to wait"]
data = [0, 0, 0]

def func(d):
    val = d['When looking for the equipment you want to use, is it available?']
    if not (type(val) == float and math.isnan(val)):
      index = labels.index(val)
      data[index] = data[index] + 1

# Populate Male_D and Female_D
df.apply(func, axis=1)

# Create the figure
fig, (ax1) = plt.subplots(figsize=(15, 8))
fig.suptitle('When looking for the equipment you want to use, is it available?')
fig.tight_layout()

# Create the graph's and plot
plotdata = pd.DataFrame({ "data":data, "labels":labels })
plotdata.plot.pie(y='data', labels=labels, title="", legend=True, autopct='%1.1f%%', explode=(0, 0, 0.1), shadow=True, startangle=0, ax=ax1, )


fig.show()



***Graph*** \
**Overlay Model**  \
**Time of Day vs Gym Congestion** \
*Independent vs Dependent Data*  

Goal of this graph is to chart what times of days certian zones of the gym tend to be the most congested \
This will then be releated to the DesReq chart for scheduled gym zones.

Time Series graph

**Previous Cells Required!!**  \
*Male vs Female Gym Zone Usage*  \
*Male vs Female Average Daily Swipe Time*




In [ ]:

# There are 7 zones
# I want to make a line for each zone

times = ['06AM', '07AM', '08AM', '09AM', '10AM', '11AM', '12PM', '01PM', '02PM', '03PM', '04PM', '05PM', '06PM', '07PM', '08PM', '09PM', '10PM']
zone_map = ['Cardio (front room)',	'Cables Area (middle room)',	'Weight Room (back room)', 'Studio 1 (front)', 'Spin Studio',	'Studio 3 (located in Weight Room)',	'Indoor Track']

# Create the function x to define how congested a zone is at a time
def x(m_df, f_df, zone):

  # Find the congestion over every hour for Males and Females
  # Pre-defined in Variables
  # male_congestion_data
  # female_congestion_data

  vals = []

  # Loop over all of the dates
  for time in range(len(times)):

    # Formula represents the swipe time of males and females in relation to the congestion that a Male or Female brings to a zone
    # Formula = (m_df[time] * male_congestion_data[zone]) + (f_df[time] + female_congestion_data[zone])
    Formula = (m_df[time] * male_congestion_data[zone]) + (f_df[time] * female_congestion_data[zone])
    total = Formula
    #swipe_count = m_df[time] + f_df[time]
    #congestion_level = male_congestion_data[zone] + female_congestion_data[zone]
    #total = swipe_count * congestion_level
    vals.append(total)

  return vals

# Plot information
plt.tight_layout() # Assign a tigh layout for the plots
# Plot the averages constructed
fig, (ax1, ax2, ax3, ax4, ax5, ax6, ax7) = plt.subplots(7, figsize=(15, 45))
fig.suptitle(f"{season} Overlay Model")
fig.tight_layout()

ax1.set_title("Monday")
ax2.set_title("Tuseday")
ax3.set_title("Wednesday")
ax4.set_title("Thursday")
ax5.set_title("Friday")
ax6.set_title("Saturday")
ax7.set_title("Sunday")

axis = [ax1, ax2, ax3, ax4, ax5, ax6, ax7]

for a in range(len(axis)):
  # Append all Weekday Male Averages to the graph
  day = a # Monday
  res = []
  for zone in range(len(zone_map)):
    # Find the number of swipes over every hour for Male and Females
    swipe_count_male = [Male_Swipe[day][x] if x in Male_Swipe[day] else 0.0 for x in dates]
    swipe_count_female = [Female_Swipe[day][x] if x in Female_Swipe[day] else 0.0 for x in dates]
    vals = x(swipe_count_male, swipe_count_female, zone)
    res.append(vals)
    axis[a].plot(times, vals, '-', label=zone_map[zone]) # this will show date at the x-axis
  #axis[a].stackplot(times, res[0], res[1], res[2], res[3], res[4], res[5], res[6], labels=zone_map) # this will show date at the x-axis

# Legend
ax1.legend()
ax2.legend()
ax3.legend()
ax4.legend()
ax5.legend()
ax6.legend()
ax7.legend()

fig.show()



**Questions and Responses**

In [ ]:
number_of_male_swipes = i_data['Gender'].value_counts()['Male']
number_of_female_swipes = i_data['Gender'].value_counts()['Female']

print(number_of_male_swipes)
print(number_of_female_swipes)
print(f"Ratio of Male to Female swipes {number_of_male_swipes/number_of_female_swipes}x")

# Assuming your DataFrame is named 'df'
# If your 'Date' column is not already in datetime format, you can convert it
# Now, let's create the pie chart
gender_counts = i_data['Gender'].value_counts()
plt.figure(figsize=(8, 6))
plt.pie(gender_counts, labels=gender_counts.index, autopct='%1.1f%%', startangle=140)
plt.title('Distribution of Gender')
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.show()

i_data